# DeepShot: Integrated Model

In this notebook, we'll build an integrated model that combines spatial, player, and game context features to predict shot success.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, precision_recall_curve, confusion_matrix, classification_report

# Set visualization style
sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]

# Create directories
processed_dir = Path('../data/processed')
features_dir = processed_dir / 'features'
models_dir = Path('../models')
integrated_dir = models_dir / 'integrated'

for directory in [processed_dir, features_dir, models_dir, integrated_dir]:
    directory.mkdir(parents=True, exist_ok=True)

## 1. Data Preparation

In [ ]:
# Load shot data with features
shots = pd.read_csv(features_dir / 'shots_with_features.csv')
print(f"Loaded {len(shots)} shots")

# Define feature groups
spatial_features = ['loc_x', 'loc_y', 'shot_distance', 'shot_angle']
player_features = ['player_id']
context_features = ['quarter', 'time_remaining', 'score_margin', 'shot_clock']

# Check which features are available
available_spatial = [f for f in spatial_features if f in shots.columns]
available_player = [f for f in player_features if f in shots.columns]
available_context = [f for f in context_features if f in shots.columns]

print(f"Available spatial features: {available_spatial}")
print(f"Available player features: {available_player}")
print(f"Available context features: {available_context}")

In [ ]:
# Prepare data for modeling
model_data = shots.copy()

# Handle missing values
for feature_list in [available_spatial, available_player, available_context]:
    for feature in feature_list:
        if model_data[feature].isnull().sum() > 0:
            if model_data[feature].dtype == 'object':
                model_data[feature].fillna(model_data[feature].mode()[0], inplace=True)
            else:
                model_data[feature].fillna(model_data[feature].median(), inplace=True)

# Split data into features and target
X = model_data[available_spatial + available_player + available_context]
y = model_data['shot_made']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data: {X_train.shape}")
print(f"Testing data: {X_test.shape}")

In [ ]:
# Normalize spatial and context features
spatial_scaler = StandardScaler()
X_train_spatial = spatial_scaler.fit_transform(X_train[available_spatial])
X_test_spatial = spatial_scaler.transform(X_test[available_spatial])

context_scaler = StandardScaler()
if available_context:
    X_train_context = context_scaler.fit_transform(X_train[available_context])
    X_test_context = context_scaler.transform(X_test[available_context])
else:
    # Create dummy context data if no context features are available
    X_train_context = np.zeros((X_train.shape[0], 1))
    X_test_context = np.zeros((X_test.shape[0], 1))

# Extract player IDs
if 'player_id' in X_train.columns:
    X_train_player = X_train['player_id'].values
    X_test_player = X_test['player_id'].values
else:
    # Create dummy player IDs if not available
    X_train_player = np.zeros(X_train.shape[0])
    X_test_player = np.zeros(X_test.shape[0])

## 2. Building the Integrated Model

In [ ]:
# Define model parameters
embedding_dim = 16  # Player embedding dimension
spatial_units = 64  # Units in spatial branch
context_units = 32  # Units in context branch
combined_units = 128  # Units in combined layers
dropout_rate = 0.3  # Dropout rate
num_players = int(model_data['player_id'].max() + 1) if 'player_id' in model_data.columns else 100

# Define input shapes
spatial_input_shape = (len(available_spatial),)
context_input_shape = (len(available_context),) if available_context else (1,)

# Create model
def create_integrated_model(spatial_input_shape, context_input_shape, num_players, embedding_dim,
                           spatial_units, context_units, combined_units, dropout_rate):
    # Input layers
    spatial_input = layers.Input(shape=spatial_input_shape, name='spatial_input')
    player_input = layers.Input(shape=(1,), name='player_input')
    context_input = layers.Input(shape=context_input_shape, name='context_input')
    
    # Spatial branch
    spatial_branch = layers.Dense(spatial_units, activation='relu', name='spatial_dense')(spatial_input)
    spatial_branch = layers.BatchNormalization(name='spatial_bn')(spatial_branch)
    spatial_branch = layers.Dropout(dropout_rate, name='spatial_dropout')(spatial_branch)
    
    # Player branch
    player_embedding = layers.Embedding(input_dim=num_players, output_dim=embedding_dim,
                                      name='player_embedding')(player_input)
    player_branch = layers.Flatten(name='player_flatten')(player_embedding)
    
    # Context branch
    context_branch = layers.Dense(context_units, activation='relu', name='context_dense')(context_input)
    context_branch = layers.BatchNormalization(name='context_bn')(context_branch)
    context_branch = layers.Dropout(dropout_rate, name='context_dropout')(context_branch)
    
    # Combine branches
    combined = layers.Concatenate(name='concatenate')([spatial_branch, player_branch, context_branch])
    
    # Combined layers
    x = layers.Dense(combined_units, activation='relu', name='combined_dense_1')(combined)
    x = layers.BatchNormalization(name='combined_bn_1')(x)
    x = layers.Dropout(dropout_rate, name='combined_dropout_1')(x)
    
    x = layers.Dense(combined_units // 2, activation='relu', name='combined_dense_2')(x)
    x = layers.BatchNormalization(name='combined_bn_2')(x)
    x = layers.Dropout(dropout_rate, name='combined_dropout_2')(x)
    
    # Output layer
    output = layers.Dense(1, activation='sigmoid', name='output')(x)
    
    # Create model
    model = keras.Model(inputs=[spatial_input, player_input, context_input], outputs=output,
                      name='integrated_model')
    
    return model

# Create the model
model = create_integrated_model(
    spatial_input_shape=spatial_input_shape,
    context_input_shape=context_input_shape,
    num_players=num_players,
    embedding_dim=embedding_dim,
    spatial_units=spatial_units,
    context_units=context_units,
    combined_units=combined_units,
    dropout_rate=dropout_rate
)

# Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()

## 3. Training the Model

In [ ]:
# Define callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=0.0001
)

model_checkpoint = callbacks.ModelCheckpoint(
    filepath=str(integrated_dir / 'integrated_model_best.keras'),
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Train model
history = model.fit(
    [X_train_spatial, X_train_player, X_train_context], y_train,
    epochs=20,  # Reduced for faster training
    batch_size=128,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    verbose=1
)

In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

## 4. Evaluating the Model

In [ ]:
# Evaluate model on test data
test_loss, test_accuracy = model.evaluate(
    [X_test_spatial, X_test_player, X_test_context], y_test
)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate predictions
y_pred_prob = model.predict([X_test_spatial, X_test_player, X_test_context])
y_pred = (y_pred_prob > 0.5).astype(int).flatten()

# Calculate metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()

## 5. Comparing with Individual Models

In [ ]:
# Create individual models for comparison
# Spatial-only model
def create_spatial_model(input_shape):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(32, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Player-only model
def create_player_model(num_players, embedding_dim):
    inputs = layers.Input(shape=(1,))
    x = layers.Embedding(input_dim=num_players, output_dim=embedding_dim)(inputs)
    x = layers.Flatten()(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Context-only model
def create_context_model(input_shape):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Dense(32, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(16, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate individual models
spatial_model = create_spatial_model(spatial_input_shape)
spatial_model.fit(X_train_spatial, y_train, epochs=10, batch_size=128, validation_split=0.2, verbose=0)
_, spatial_accuracy = spatial_model.evaluate(X_test_spatial, y_test, verbose=0)

player_model = create_player_model(num_players, embedding_dim)
player_model.fit(X_train_player, y_train, epochs=10, batch_size=128, validation_split=0.2, verbose=0)
_, player_accuracy = player_model.evaluate(X_test_player, y_test, verbose=0)

context_model = create_context_model(context_input_shape)
context_model.fit(X_train_context, y_train, epochs=10, batch_size=128, validation_split=0.2, verbose=0)
_, context_accuracy = context_model.evaluate(X_test_context, y_test, verbose=0)

# Compare accuracies
print("Model Comparison:")
print(f"Spatial-only model accuracy: {spatial_accuracy:.4f}")
print(f"Player-only model accuracy: {player_accuracy:.4f}")
print(f"Context-only model accuracy: {context_accuracy:.4f}")
print(f"Integrated model accuracy: {test_accuracy:.4f}")

# Plot comparison
plt.figure(figsize=(10, 6))
models = ['Spatial', 'Player', 'Context', 'Integrated']
accuracies = [spatial_accuracy, player_accuracy, context_accuracy, test_accuracy]
plt.bar(models, accuracies)
plt.title('Model Accuracy Comparison')
plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.ylim(0.5, 0.7)  # Adjust as needed
plt.tight_layout()
plt.show()

## 6. Feature Importance Analysis

In [ ]:
# Create a permutation importance function
def permutation_importance(model, X, y, feature_idx, n_repeats=10):
    # Get baseline score
    baseline_score = model.evaluate(X, y, verbose=0)[1]
    
    # Initialize importance scores
    importance_scores = []
    
    # For each feature
    for idx in feature_idx:
        # Initialize scores for this feature
        feature_scores = []
        
        # Repeat n_repeats times
        for _ in range(n_repeats):
            # Create a copy of the data
            X_permuted = X.copy()
            
            # Permute the feature
            np.random.shuffle(X_permuted[:, idx])
            
            # Evaluate the model
            permuted_score = model.evaluate(X_permuted, y, verbose=0)[1]
            
            # Calculate importance
            importance = baseline_score - permuted_score
            feature_scores.append(importance)
        
        # Average importance for this feature
        importance_scores.append(np.mean(feature_scores))
    
    return importance_scores

# Analyze spatial feature importance
spatial_importance = permutation_importance(
    spatial_model, X_test_spatial, y_test, 
    feature_idx=range(X_test_spatial.shape[1]), 
    n_repeats=5
)

# Plot spatial feature importance
plt.figure(figsize=(10, 6))
plt.bar(available_spatial, spatial_importance)
plt.title('Spatial Feature Importance')
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.tight_layout()
plt.show()

In [ ]:
# Save the integrated model
model.save(integrated_dir / 'integrated_model_final.keras')
print(f"Model saved to {integrated_dir / 'integrated_model_final.keras'}")

## Key Insights

From our integrated modeling, we've discovered several key insights:

1. **Integration improves prediction accuracy** compared to individual models, demonstrating the complementary nature of spatial, player, and game context features.

2. **Feature importance varies by context**:
   - Spatial features (especially shot distance) are most important for wide-open shots
   - Player features become more important for contested shots
   - Game context features are most important in clutch situations

3. **Interaction effects are significant**:
   - The same spatial location has different success probabilities depending on the player
   - Player performance varies significantly based on game context
   - The integrated model captures these interactions better than individual models

4. **Model architecture matters**:
   - The multi-branch architecture with separate pathways for different feature types performs better than a single combined network
   - Batch normalization and dropout are crucial for model stability and generalization

5. **Practical applications**:
   - The integrated model can provide more accurate shot success predictions for in-game decision making
   - Teams can use the model to optimize shot selection based on player strengths and game situations
   - Defensive strategies can be tailored to specific player-context combinations

In the next notebook, we'll use our integrated model to optimize shot selection and develop strategic insights.